# Reader - Implantação

Este componente utiliza um modelo de QA pré-treinado em Português com o dataset SQuAD v1.1, é um modelo de domínio público disponível em [Hugging Face](https://huggingface.co/pierreguillou/bert-large-cased-squad-v1.1-portuguese).<br>

Seu objetivo é encontrar a resposta de uma ou mais perguntas de acordo com uma lista de contextos distintos.

A tabela de dados de entrada deve possuir uma coluna de contextos, em que cada linha representa um contexto diferente, e uma coluna de perguntas em que cada linha representa uma pergunta a ser realizada. Note que para cada pergunta serão utilizados todos os contextos fornecidos para realização da inferência, e portanto, podem haver bem mais contextos do que perguntas.

Obs: Este componente utiliza recursos da internet, portanto é importante estar conectado à rede para que este componente funcione corretamente.<br>
### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo-real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [1]:
%%writefile Model.py

import joblib
import numpy as np
import pandas as pd
from reader import Reader

class Model:
    def __init__(self):
        self.loaded = False

    def load(self):
        
        # Load artifacts
        artifacts = joblib.load("/tmp/data/reader.joblib")
        self.model_parameters = artifacts["model_parameters"]
        self.inference_parameters = artifacts["inference_parameters"]
        
        # Initialize reader
        self.reader = Reader(**self.model_parameters)
        
        # Set model loaded
        self.loaded = True
        print("Loaded model")
    
    def class_names(self):
        column_names = list(self.inference_parameters['output_columns'])
        return column_names

    def predict(self, X, feature_names, meta=None):
        if not self.loaded:
            self.load()
            
        # Convert to dataframe
        if feature_names != []:
            df = pd.DataFrame(X, columns = feature_names)
            df = df[self.inference_parameters['input_columns']]
        else:
            df = pd.DataFrame(X, columns = self.inference_parameters['input_columns'])
            
        # Predict answers #

        # Iterate over dataset
        for idx, row in df.iterrows():

            # Get question
            question = row[self.inference_parameters['question_column_name']]

            # Get context
            context = row[self.inference_parameters['context_column_name']]

            # Make prediction
            answer, probability, _ = self.reader([question], [context])

            # Save to df
            df.at[idx, self.inference_parameters['answer_column_name']] = answer[0]
            df.at[idx, self.inference_parameters['proba_column_name']] = probability[0]

        # Retrieve Only Best Answer #

        # Initializate best df
        best_df = pd.DataFrame(columns=df.columns)

        # Get unique questions
        unique_questions = df[self.inference_parameters['question_column_name']].unique()

        # Iterate over each unique question
        for question in unique_questions:

            # Filter df
            question_df = df[df[self.inference_parameters['question_column_name']] == question]

            # Sort by score (descending)
            question_df = question_df.sort_values(by=self.inference_parameters['proba_column_name'], ascending=False).reset_index(drop=True)

            # Append best ansewer to output df
            best_df = pd.concat((best_df,pd.DataFrame(question_df.loc[0]).T)).reset_index(drop=True)
            
        if self.inference_parameters['keep_best'] == 'sim':
            return best_df.values
        else:
            return df.values

Overwriting Model.py
